https://www.guru99.com/linear-regression-tensorflow.html

# 0 - Setup the environment

In [1]:
import pandas as pd
from sklearn import datasets
import tensorflow as tf
import itertools

# 1 - Import the data

In [2]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
training_set = pd.read_csv("boston_train/boston_train.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

test_set = pd.read_csv("boston_train/boston_test.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

prediction_set = pd.read_csv("boston_train/boston_predict.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

In [3]:
print(training_set.shape, test_set.shape, prediction_set.shape)


(400, 10) (100, 10) (6, 10)


In [4]:
FEATURES = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio"]
LABEL = "medv"

# 2 - Convert the data

In [5]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

# 3 - Define the estimator

In [6]:
estimator = tf.estimator.LinearRegressor(    
        feature_columns=feature_cols,   
        model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
tf.keras.backend.set_floatx('float64')
def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):    
         return  tf.compat.v1.estimator.inputs.pandas_input_fn(       
         x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),       
         y = pd.Series(data_set[LABEL].values),       
         batch_size=n_batch,          
         num_epochs=num_epochs,       
         shuffle=shuffle)

# 4 - Train the model

In [8]:
estimator.train(input_fn=get_input_fn(training_set,                                       
                                           num_epochs=None,                                      
                                           n_batch = 128,                                      
                                           shuffle=False),                                      
                                           steps=1000)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-3000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct

# 5 - Evaluate your model

In [9]:
ev = estimator.evaluate(    
          input_fn=get_input_fn(test_set,                          
          num_epochs=1,                          
          n_batch = 128,                          
          shuffle=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-11-02T12:43:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.55309s
INFO:tensorflow:Finished evaluation at 2020-11-02-12:43:04
INFO:tensorflow:Saving dict for global step 4000: average_loss = 20.341147933350875, global_step = 4000, label/mean = 22.079999999999995, loss = 20.341148, prediction/mean = 22.887005138397218
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: train/model.ckpt-4000


In [12]:
loss_score = ev["loss"]

print("Loss: {0:f}".format(loss_score))

print(ev)

Loss: 20.341148
{'average_loss': 20.341147933350875, 'label/mean': 22.079999999999995, 'loss': 20.341148, 'prediction/mean': 22.887005138397218, 'global_step': 4000}


In [34]:
training_set['medv'].describe()

count    400.000000
mean      22.625500
std        9.572593
min        5.000000
25%       16.600000
50%       21.400000
75%       25.025000
max       50.000000
Name: medv, dtype: float64

# Make the prediction

In [35]:
y = estimator.predict(    
         input_fn=get_input_fn(prediction_set,                          
         num_epochs=1,                          
         n_batch = 128,                          
         shuffle=False))

In [36]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([33.803967], dtype=float32), array([18.942839], dtype=float32), array([26.030296], dtype=float32), array([31.343998], dtype=float32), array([15.66669], dtype=float32), array([20.952637], dtype=float32)]
